In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd 
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import StackingRegressor
import xgboost as xgb

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Loading Data

In [2]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Pre-processing and Feature Engineering

In [3]:
def addFeatures(df):
    
    df['YrBltAndRemod'] = df['YearBuilt'] + df['YearRemodAdd']
    df['TotalSF'] = df['TotalBsmtSF'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['Total_sqr_footage'] = df['BsmtFinSF1'] + df['BsmtFinSF2'] + df['1stFlrSF'] + df['2ndFlrSF']
    df['Total_Bathrooms'] = df['FullBath'] + (0.5 * df['HalfBath']) + df['BsmtFullBath'] + (0.5 * df['BsmtHalfBath'])
    df['Total_porch_sf'] = df['OpenPorchSF'] + df['3SsnPorch'] + df['EnclosedPorch'] + df['ScreenPorch'] + df['WoodDeckSF']

    return df

def preprocess(df):

    df['LotFrontage'] = df.groupby('Neighborhood')['LotFrontage'].transform(lambda x : x.mean())    
    
    for col in df.select_dtypes(exclude='number').columns:
        df[col] = df[col].fillna('None')

    for col in df.select_dtypes(include='number').columns:
        df[col] = df[col].fillna(0)

    return df

def encoding(df):
    ohe = OneHotEncoder(sparse_output = False, drop = 'first')
    catagorical_cols = list(df.select_dtypes(exclude='number').columns)
    numaric_cols = list(df.select_dtypes(include='number').columns)
 
    encoded_values = ohe.fit_transform(df[catagorical_cols])
    encoded_cols = list(ohe.get_feature_names_out())
    df[encoded_cols] = encoded_values


    return df[encoded_cols + numaric_cols]

There are some outliers in the training dataset. You can drop them, but I'll leave them for you to experiment with.

In [4]:
# Outliers
# train_df = train_df.drop(index=[30, 88, 462, 631, 1322, 523, 691, 1182, 1298]).reset_index(drop=True)

Here, I'm concatenating the training and test datasets to preprocess them.

In [5]:
all_df = pd.concat([train_df, test_df])

index = train_df.shape[0]
target = all_df.iloc[:index]['SalePrice']

all_df = addFeatures(all_df)
all_df = preprocess(all_df)
all_df = encoding(all_df)
id = all_df['Id']
all_df.drop(columns=['Id', 'SalePrice'], inplace= True)

C:\Users\Lario\AppData\Local\Temp\ipykernel_21540\400488452.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[encoded_cols] = encoded_values
C:\Users\Lario\AppData\Local\Temp\ipykernel_21540\400488452.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[encoded_cols] = encoded_values
C:\Users\Lario\AppData\Local\Temp\ipykernel_21540\400488452.py:30: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum

# Training


I'm using XGBoost and Ridge Regressor, then Linear Regression to stack them. You can go crazy and add more. I've used Optuna to tune the XGBoost regressor.

In [6]:
xgb_params = {'max_depth': 4, 'learning_rate': 0.00875, 'n_estimators': 3515, 'min_child_weight': 2, 'colsample_bytree': 0.2050378195385253, 'subsample': 0.40369887914955715, 'reg_alpha': 0.3301567121037565, 'reg_lambda': 0.046181862052743}

xgbr = xgb.XGBRegressor(**xgb_params, random_state=42)

model = [
    ('ridge', Ridge(alpha=15)),
    ('XGB', xgbr),
] 

stack = StackingRegressor(estimators=model, final_estimator=LinearRegression())

stack.fit(all_df.iloc[:index], np.log1p(target.values))

,"estimators estimators: list of (str, estimator)Base estimators which will be stacked together. Each element of thelist is defined as a tuple of string (i.e. name) and an estimatorinstance. An estimator can be set to 'drop' using `set_params`.","[('ridge', ...), ('XGB', ...)]"
,"final_estimator final_estimator: estimator, default=NoneA regressor which will be used to combine the base estimators.The default regressor is a :class:`~sklearn.linear_model.RidgeCV`.",LinearRegression()
,"cv cv: int, cross-validation generator, iterable, or ""prefit"", default=NoneDetermines the cross-validation splitting strategy used in`cross_val_predict` to train `final_estimator`. Possible inputs forcv are:* None, to use the default 5-fold cross validation,* integer, to specify the number of folds in a (Stratified) KFold,* An object to be used as a cross-validation generator,* An iterable yielding train, test splits,* `""prefit""`, to assume the `estimators` are prefit. In this case, the estimators will not be refitted.For integer/None inputs, if the estimator is a classifier and y iseither binary or multiclass,:class:`~sklearn.model_selection.StratifiedKFold` is used.In all other cases, :class:`~sklearn.model_selection.KFold` is used.These splitters are instantiated with `shuffle=False` so the splitswill be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here.If ""prefit"" is passed, it is assumed that all `estimators` havebeen fitted already. The `final_estimator_` is trained on the `estimators`predictions on the full training set and are **not** cross validatedpredictions. Please note that if the models have been trained on the samedata to train the stacking model, there is a very high risk of overfitting... versionadded:: 1.1 The 'prefit' option was added in 1.1.. note:: A larger number of split will provide no benefits if the number of training samples is large enough. Indeed, the training time will increase. ``cv`` is not used for model evaluation but for prediction.",None
,"n_jobs n_jobs: int, default=NoneThe number of jobs to run in parallel for `fit` of all `estimators`.`None` means 1 unless in a `joblib.parallel_backend` context. -1 meansusing all processors. See :term:`Glossary ` for more details.",None
,"passthrough passthrough: bool, default=FalseWhen False, only the predictions of estimators will be used astraining data for `final_estimator`. When True, the`final_estimator` is trained on the predictions as well as theoriginal training data.",False
,"verbose verbose: int, default=0Verbosity level.",0
,"alpha alpha: {float, ndarray of shape (n_targets,)}, default=1.0Constant that multiplies the L2 term, controlling regularizationstrength. `alpha` must be a non-negative float i.e. in `[0, inf)`.When `alpha = 0`, the objective is equivalent to ordinary leastsquares, solved by the :class:`LinearRegression` object. For numericalreasons, using `alpha = 0` with the `Ridge` object is not advised.Instead, you should use the :class:`LinearRegression` object.If an array is passed, penalties are assumed to be specific to thetargets. Hence they must correspond in number.",15
,"fit_intercept fit_intercept: bool, default=TrueWhether to fit the intercept for this model. If setto false, no intercept will be used in calculations(i.e. ``X`` and ``y`` are expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"max_iter max_iter: int, default=NoneMaximum number of iterations for conjugate gradient solver.For 'sparse_cg' and 'lsqr' solvers, the default value is determinedby scipy.sparse.linalg. For 'sag' solver, the default value is 1000.For 'lbfgs' solver, the default value is 15000.",None
,"tol tol: float, default=1e-4The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for each solver:- 'svd': `tol` has no impact.- 'cholesky': `tol` has no impact.- 'sparse_cg': 

# Output

In [7]:
predict = stack.predict(all_df)

output = pd.DataFrame({
    'Id' : id[index:],
    'SalePrice' : np.exp(predict[index:])
})

output.to_csv('submission.csv' ,index=False)

This should give you a decent score and place you in the top 2%, but we want to be in the top 1%, don't we? There is a data leakage, and since we have to brag to our friends, we should exploit it. It's Kaggle, after all.

# Exploiting Data Leakage (Optional)

In [10]:
train = pd.read_csv('data/AmesHousing.csv')
train.drop(['PID'], axis=1, inplace=True)

origin = pd.read_csv('data/train.csv')
train.columns = origin.columns

test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [11]:
missing = test.isnull().sum()
missing = missing[missing>0]
train.drop(missing.index, axis=1, inplace=True)
train.drop(['Electrical'], axis=1, inplace=True)

test.dropna(axis=1, inplace=True)
test.drop(['Electrical'], axis=1, inplace=True)

In [12]:
for i in range(len(test)):
    for j in range(0, len(train)):
        for k in range(1, len(test.columns)):
            if test.iloc[i,k] == train.iloc[j,k]:
                continue
            else:
                break
        else:
            submission.iloc[i, 1] = train.iloc[j, -1]
            break

In [13]:
submission.to_csv('leak-submission.csv', index=False)